In [1]:
from nltk import *
from collections import defaultdict
import string
import re,sys,pickle,math
import numpy as np
import pandas as pd
from nltk.corpus import wordnet

In [2]:
path_train = "train.csv"
train = pd.read_csv(path_train)
train.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


In [3]:
def word_net(word):
    """
    Aims at lemmatizing a word using synonyms of a word. 
    Longest prefix match of a word with some constraints is the lemmatized word.
    Synonyms of a word are extracted using synsets()
    """
    if(len(word)<=5):
        return word
    synonyms = []
    #antonyms = []
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonyms.append(l.name())
            #if l.antonyms():
                #antonyms.append(l.antonyms()[0].name())
    best_match = ""
    match = 0
    if len(synonyms)==0:
        return word
    left_count = len(word)
    for syn in synonyms:
        if syn==word:
            continue
        temp = 0
        flag=1
        index = 0
        for i in range(min(len(syn),len(word))):
            index = i
            if(syn[i]!=word[i]):
                if((len(syn)-temp)<left_count):
                    left_count = len(syn)-temp
                    best_match = syn 
                    match = i
                flag=0
                break
            temp+=1
        if(flag==1):
            if((len(syn)-temp)<left_count):
                left_count = len(syn)-temp
                best_match = syn
                match = index+1
    if(len(word)-match>4 or len(best_match)>=len(word) or len(best_match)<=1):
        return word
    return best_match

def write_pkl(file,data):
    """
    Writes data from a pickle file. ie. "data.pkl"
    """
    writing = open(file, 'wb')
    pickle.dump(data, writing)
    return

def load_pkl(file):
    """
    Loads data from a pickle file. ie. "data.pkl"
    """
    pkl_file = open(file, 'rb')
    data = pickle.load(pkl_file)
    return data

corpus_size = 38932

def tf(word,doc_id):
    """
    Calculates and returns the term frequency log(tf+1)
    """
    try:
        return math.log(multimap_list[word][doc_id][1]+1)
    except:
        return 0

def idf(word):
    """
    Calculaltes and returns the inverse document frequency log(N/df)
    """
    try:
        return math.log(float(corpus_size)/len(multimap_list[word]))
    except:
        return 0

In [4]:
desc = np.array(train["Description"])
print(desc[1],desc.shape)

I stayed at the Crown Plaza April -- - April --, ----. The staff was friendly and attentive. The elevators are tiny (about -' by -'). The food in the restaurant was delicious but priced a little on the high side. Of course this is Washington DC. There is no pool and little for children to do. My room on the fifth floor had two comfortable beds and plenty of space for one person. The TV is a little small by todays standards with a limited number of channels. There was a small bit of mold in the bathtub area that could have been removed with a little bleach. It appeared the carpets were not vacummed every day. I reported a light bulb was burned out. It was never replaced. Ice machines are on the odd numbered floors, but the one on my floor did not work. I encountered some staff in the elevator one evening and I mentioned the ice machine to them. Severel hours later a maid appeared at my door with ice and two mints. I'm not sure how they knew what room I was in. That was a little unnervin

In [5]:
"""
#cleaning the data 
#removing punctuations
#to lowercase
#remove stop words from file

clean_train = []
with open("stp.txt") as file:
    stops = file.read().splitlines()
stops = set(stops)
for review in desc:
    review = re.sub(r'[^A-Za-z0-9\s]',r'',review)
    review = re.sub(r'\n',r' ',review)
    review = " ".join([w.lower() for w in review.split()])
    #review = " ".join([word_net(w) for w in review.split()])
    review = " ".join([word_net(w) for w in review.split() if w not in stops])
    clean_train.append(review)
    
write_pkl("data.pkl",clean_train)
"""
print("NOTE: This was carried out and stored in data.pkl")

NOTE: This was carried out and stored in data.pkl


In [6]:
#Loading the cleaned training data
clean_train = load_pkl("data.pkl")
print(clean_train[1])
print(len(clean_train))

stay crown plaza april april staff was friendly attentive elevator are tiny food restaurant was delicious but price little on high side of course is washington dc is no pool little child to do room on fifth floor had two comfortable beds plenty of space one person tv is little small today standard limit number of channel was small bit of mold bath area could have been remove little bleach appear carpet were not vacummed every day report light bulb was burn was never replace ice machine are on odd number floor but one on floor did not work encounter staff elevator one eve mention ice machine to severel hours later maid appear door ice two mints im not sure knew room was was little unnerve would stay again business but would not come on vacation
38932


In [7]:
#loading the result happy or not into array
train_result = np.array(np.where((train["Is_Response"]=="not happy"), 0, 1))
#train_result = np.array(train["Is_Response"])
print(train_result[:10],len(train_result))
print(train["Is_Response"].head(10))
#write_pkl("train_result.pkl",train_result)

[0 0 0 1 0 1 0 1 1 0] 38932
0    not happy
1    not happy
2    not happy
3        happy
4    not happy
5        happy
6    not happy
7        happy
8        happy
9    not happy
Name: Is_Response, dtype: object


In [8]:
"""
# eng_dict contains all words in dictionary
with open("20k.txt") as file:
    eng_dict = set(file.read().split())

# cleaning combined words and misspellings
def corrected(word):
    for i in range(len(word)):
        if word[:i+1] in eng_dict and word[i+1:] in eng_dict:
            return i
    return -1

count = 0
words = set([])
for review in clean_train:
    if(count%1000==0):
        print("Review no",count+1)
    count+=1
    for word in review.split():
        if word not in eng_dict:
            index = corrected(word)
            if index!= -1:
                print(word[:index+1]+"-"+word[index+1:],end=" ")
                words.add(word[:index+1])
                words.add(word[index+1:])
            else:
                words.add(word)
print(list(words)[:50],len(words))
"""
print("Needs Work!!")

Needs Work!!


In [9]:
"""
#indexing
multimap_list = {}
doc_id = 0
for review in clean_train:
    doc_id+=1
    pos = 0
    for word in review.split():
        pos+=1
        try:
            multimap_list[word][doc_id][0].append(pos)
            multimap_list[word][doc_id][1]+=1
        except:
            try:
                multimap_list[word][doc_id]=[[],[]]
                multimap_list[word][doc_id][0]=[]
                multimap_list[word][doc_id][1]=0
                multimap_list[word][doc_id][0].append(pos)
                multimap_list[word][doc_id][1]+=1
            except:
                multimap_list[word]={}
                multimap_list[word][doc_id]=[[],[]]
                multimap_list[word][doc_id][0]=[]
                multimap_list[word][doc_id][1]=0
                multimap_list[word][doc_id][0].append(pos)
                multimap_list[word][doc_id][1]+=1
    if(doc_id%1000==0):
        print("length so far",len(multimap_list))
        
writing = open('multimap.pkl', 'wb')
pickle.dump(multimap_list, writing)
"""
print("Done indexing! Load it from multimap.pkl")

Done indexing! Load it from multimap.pkl


In [10]:
#Loading the indexed multimap.pkl file
multimap_list = load_pkl("multimap.pkl")
print(multimap_list["drone"])

{30051: [[16], 1], 14951: [[152], 1], 35370: [[84], 1], 7409: [[176], 1], 30892: [[488], 1], 32305: [[45], 1], 24405: [[18], 1], 33847: [[51], 1], 13273: [[34], 1], 26939: [[24], 1], 27761: [[95], 1], 20287: [[65], 1]}


In [11]:
print(clean_train[21540-1])

couple felt comfortable return to hotel than once whenever need super affordable place to stay downtown sf age of hotel consider skyhigh price of most hotel area regardless of age was gem havent seen place since but stay least week once completely satisfy lived sf area time knew was very reasonable choice is locate on post street is literally few block everything downtown sf has to offer short walk to famed cable cars right on top of bart station come sfo airport union square great place to eat loads of public transportation hotel is also situate bottom of nob hill is very little inclination to street no steep hills to worry most cases stay decent part of downtown nob hill will require hike up crazy steep hill to access place not case dakota very easy to access convenient little market surround neighborhood last min needs also atm late night snack few block dakota sits is not most spark clean area of downtown however is securely locate few street up most sketchy part tenderloin actuall

In [42]:
#precalc tfidf for all words for all docs
corpus_size = 38932

def tf(word,doc_id):
    """
    Calculates and returns the term frequency log(tf+1)
    """
    try:
        return math.log(multimap_list[word][doc_id][1]+1)
    except:
        return 0

def idf(word):
    """
    Calculaltes and returns the inverse document frequency log(N/df)
    """
    try:
        return math.log(float(corpus_size)/len(multimap_list[word]))
    except:
        return 0
"""
tfidf = load_pkl("multimap.pkl")
words = []
for i in multimap_list:
    words.append(i)

for word in words[:]:
    for doc_id in multimap_list[word]:
        tfidf[word][doc_id][1] = tf(word,doc_id)*idf(word)
    
write_pkl("tfidf.pkl",tfidf)
"""
print("Ran and calculated tfidf!")

Ran and calculated tfidf!
